# 01 - Exploración de datos

## Objetivo
Explorar los videos o frames sublinguales para evaluar:
- Calidad de imagen
- Resolución, duración
- Ejemplos con artefactos
- Variables que puedan afectar el análisis

## Dataset
Se asume que los videos están en `data/raw/` en formato `.mp4` o `.avi`


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Ruta de videos
data_path = "../data/raw"
video_files = [f for f in os.listdir(data_path) if f.endswith(('.mp4', '.avi'))]

print(f"{len(video_files)} videos encontrados.")

# Visualizar el primer frame del primer video
video = cv2.VideoCapture(os.path.join(data_path, video_files[0]))
ret, frame = video.read()
video.release()

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("Primer frame")
plt.axis("off")
plt.show()